# Practical Part: Neural Network Implementation & Experiments

Team:
* Jonathan Bhimani-Burrows ()
* Arlie Coles (20121051)
* Yue (Violet) Guo (20120727)

Load the Fashion MNIST data:
Note: keep your file structures like this for reading input data without
using ```import os``` for path change!
```
./Homework 3
├── 3_practical_part.ipynb
├── circles.txt
├── data
│   ├── fashion
│   │   ├── t10k-images-idx3-ubyte.gz
│   │   ├── t10k-labels-idx1-ubyte.gz
│   │   ├── train-images-idx3-ubyte.gz
│   │   └── train-labels-idx1-ubyte.gz
│   └── mnist
│       └── README.md
├── hw3
│   └── d3english.pdf
├── overleaf_url.txt
└── utils
    ├── __init__.py
    ├── __pycache__
    │   ├── __init__.cpython-36.pyc
    │   └── mnist_reader.cpython-36.pyc
    ├── argparser.py
    ├── helper.py
    └── mnist_reader.py
```

In [3]:
import utils.mnist_reader as mnist_reader
import numpy as np
import math
import copy 

In [4]:
X_train, y_train = mnist_reader.load_mnist('data/fashion', kind='train')
X_test, y_test = mnist_reader.load_mnist('data/fashion', kind='t10k')

print(X_test.shape)

FileNotFoundError: [Errno 2] No such file or directory: 'data/fashion\\train-labels-idx1-ubyte.gz'

Load the Circles data:

In [5]:
a = np.array([1, 2, 3])
b = np.array([2, 3, 4])
print(a.shape)
c= np.stack((a, b))
print(c.T)
print(c.shape)

print(c[0][0])
print(c[1][0])
print(c[0,:])

(3,)
[[1 2]
 [2 3]
 [3 4]]
(2, 3)
1
2
[1 2 3]


In [6]:
circlesData = np.loadtxt(open('circles.txt','r'))
circlesTarget = circlesData[:,2]
circlesData = circlesData[:,[0,1]] 
#circlesData = circlesData
#circlesData = np.expand_dims(circlesData, axis = 1)
#circlesData = circlesData.reshape(1100, 2, 1)
print(circlesData.shape)
circlesTarget = np.array([int(i) for i in circlesTarget])
print(circlesTarget)

(1100, 2)
[1 1 0 ... 0 1 1]


In [7]:
class loadData:
    def __init__(self):
        self.addOnes = False
        self.data_path = '/data/'
    
    def convertTarget(self, targetValues):
        # Convert to one-hot encoding
        numClasses = np.max(targetValues) + 1
        return np.eye(numClasses)[targetValues]
    

    def loadNumData(self, data, target):
        # Split into train/validation/test
        np.random.seed(6390)
        randIndices = np.random.permutation(data.shape[0])
        data, target = data[randIndices], target[randIndices]
        
        div1 = int(math.floor(0.8 * data.shape[0]))
        div2 = int(math.floor(0.9 * data.shape[0]))
        trainData, trainTarget = data[:div1], target[:div1]
        validData, validTarget = data[div1:div2], target[div1:div2]
        testData, testTarget = data[div2:], target[div2:]
    
        # Get one hot encoding
        trainTarget = self.convertTarget(trainTarget)
        validTarget = self.convertTarget(validTarget)
        testTarget = self.convertTarget(testTarget)
        
        return trainData, trainTarget, validData, validTarget, testData, testTarget

dataLoader = loadData()
 
trainData, trainTarget, validData, validTarget, testData, testTarget = dataLoader.loadNumData(circlesData, circlesTarget)
print(trainData.shape, validData.shape, testData.shape)

print(trainTarget.shape)

(880, 2) (110, 2) (110, 2)
(880, 2)


## Experiments

### Part 1

> As a beginning, start with an implementation that computes the gradients for a single example, and check that the gradient is correct using the finite difference method described above.

In [9]:
class BatchSampler(object):
    '''
    A (very) simple wrapper to randomly sample batches without replacement.
    '''
    
    def __init__(self, data, targets, batch_size):
        self.num_points = data.shape[0]
        self.features = data.shape[1]
        self.data = data
        self.targets = targets
        self.batch_size = batch_size
        self.indices = np.arange(self.num_points)

    def random_batch_indices(self, m=None):
        if m is None:
            indices = np.random.choice(self.indices, self.batch_size, replace=False)
        else:
            indices = np.random.choice(self.indices, m, replace=False)
        return indices 

    def get_batch(self, m=None):
        '''
        Get a random batch without replacement from the dataset.
        If m is given the batch will be of size m. 
        Otherwise will default to the class initialized value.
        '''
        indices = self.random_batch_indices(m)
        X_batch = np.take(self.data, indices, 0)
        y_batch = self.targets[indices]
        return X_batch, y_batch

In [10]:
# Our own activation functions

def relu(pre_activation):
    '''
    preactivation is a vector
    '''
    relu_output = np.zeros(pre_activation.shape)
    relu_flat = relu_output.flatten()
    for i, neuron in enumerate(pre_activation.flatten()):
        if neuron > 0:
            relu_flat[i] = neuron
    relu_output = relu_flat.reshape(pre_activation.shape)
    return relu_output

def softmax(pre_activation):
    '''
    Numerically stable because subtracting the max value makes bit overflow impossible,
    we will only have non-positive values in the vector
    '''
    exps = np.exp(pre_activation - np.max(pre_activation))
    return exps / np.sum(exps)

In [11]:
w1_fixed = np.array([[ 0.20960823 , 0.13663559], [ 0.38623373, -0.32807251] ,[-0.63849439 , 0.0131154 ],
                     [ 0.5142807 ,  0.0595152 ], [-0.31075243 , 0.52335846]])
w2_fixed = np.array( [[ 0.06159592, -0.10424877,  0.23591191 , 0.06177611 , 0.42799154],
                      [ 0.40780062,  0.0759027  , 0.09284926, -0.14837115 ,  0.16844463]])

In [20]:
class neuralNet():
    def __init__(self, d, dh, m, n, eta=1, regularize=None):
        self.inputDim = d #inputDim
        self.hiddenDim = dh #hiddenDim
        self.outputDim = m #outputDim
        self.regularize = regularize # lambda value
        self.learningRate = eta
        self.numData = n
        self.batchErrorGradients = []
        
        #may use xavier init - maybe explore this later.
        
        # Initial weights and biases
        self.W_1 = w1_fixed
        #np.random.uniform(-1/np.sqrt(d), 1/np.sqrt(d), d*dh).reshape(dh, d)
        self.W_2 = w2_fixed
        #np.random.uniform(-1/np.sqrt(dh), 1/np.sqrt(dh), dh*m).reshape(m, dh) 

        self.b_1 = np.zeros(dh).reshape(dh,)
        self.b_2 = np.zeros(m).reshape(m,)

        
    def fprop(self, batchData, batchTarget):
        stack_b1 = np.array([self.b_1,] * self.numData).T
        self.h_a = np.dot(self.W_1, batchData.T) + stack_b1
        print("self.h_a ", self.h_a.shape)
        # TODO: may come back to expand dim for single point
        self.h_s = relu(self.h_a)
        print("h_s relu output ", self.h_s.shape)
        stack_b2 = np.array([self.b_2,] * self.numData).T
        self.o_a = np.dot(self.W_2, self.h_s)
        print("self oa ", self.o_a.shape)
        self.o_s = softmax(self.o_a)
        print('output softmax o_s', self.o_s.shape)
        

    def errorRate(self, y, mode='matrix'):
        '''
        negative log
        -logO_s(x)
        '''        
        
        if mode == 'loop':
            negLog = -self.o_a[:,y] + np.log(np.sum(np.exp(self.o_a), axis=1))
        elif mode == 'matrix':
            print('y.shape', y.shape)
            negLog = []
            for i in range(self.numData):
                error_at_point = -self.o_a[np.argmax(y[:,i])][i] + np.log(np.sum(np.exp(self.o_a), axis=0))[i]
                negLog.append(error_at_point)
            negLog = np.array(negLog)

        return negLog
            
    def bprop(self, batchData, batchTarget):
        '''
        batchTarget already in one-hot format
        '''
        
        #batch target must be m by n
        print('os', self.o_s.shape)
        self.grad_oa = self.o_s - batchTarget
        #old_grad_oa = self.grad_oa
    
        #h_s_extend = np.repeat(self.h_s, self.outputDim ,axis = 0)  #[n*m, dh]
        #h_s_extend = np.reshape(h_s_extend,[self.numData,  self.outputDim, self.hiddenDim]) #[n,m, dh]
        #self.h_s = h_s_extend
        #self.grad_oa = np.reshape(self.grad_oa, [self.numData, self.outputDim, 1])
        print('self.grad_oa', self.grad_oa.shape)
        
        # 3D matmul??
        #self.grad_W2 = np.multiply(self.grad_oa, self.h_s)
        self.grad_W2 = np.outer(self.grad_oa, self.h_s.T)
        
        self.grad_b2 = self.grad_oa
        #self.grad_b2 = old_grad_oa
        
        self.grad_hs = np.dot(self.W_2.T , self.grad_oa)
        
        # Check this (dim mismatch maybe)
        
        #h_a_stack = np.where(self.h_a > 0, 1, 0).reshape(self.hiddenDim, self.numData, 1)
        h_a_stack = np.where(self.h_a > 0, 1, 0)
        
        # dh * n * 1, stack to dh * n *d
        #self.grad_ha = np.multiply(self.grad_hs , h_a_stack)
        self.grad_ha = np.multiply(self.grad_hs, h_a_stack)
        
        
        # grad ha = [dh, n]
        #grad_ha_extend = np.repeat(self.grad_ha, self.numData, axis = 2) #[dh*d, n]
        #print("grad_ha_extend ", grad_ha_extend.shape)
        #self.grad_ha = grad_ha_extend
        #batchData_extend = np.reshape(batchData, [self.numData, 1, self.inputDim])
        #grad_ha_extend = np.reshape(self.grad_ha, [self.numData, self.hiddenDim, self.inputDim])
        #self.grad_W1 = np.multiply(self.grad_ha, batchData_extend) #[n, d, dh]
        self.grad_W1 = np.outer(self.grad_ha, batchData)
        
        # temporary hack for grad_W1
        
        print('grad_W1', self.grad_W1)
        self.grad_b1 = self.grad_ha
        #self.grad_b1 = self.grad_b1.T

        print('grad_b1', self.grad_b1)
        
        
    
    def updateParams(self):
        #print("********in update param********")

        self.W_1 -= np.sum(self.grad_W1, axis = 0) * self.learningRate
        self.W_2 -= self.grad_W2 * self.learningRate
        #print("grad_b1 ", self.grad_b1.shape)
        
        self.b_1 -= self.grad_b1 * self.learningRate
        self.b_2 -= self.grad_b2 * self.learningRate
    
    '''
    def gradDescentLoop(self, batchData, batchTarget, K):
        # Call each example in the data (over the minibatches) in a loop
        
        
        if K == 1: # For batch size = 1 (one example)
            self.fprop(batchData, batchTarget)
            
            batchDataBack = np.expand_dims(batchData, axis = 0)
            #print('batchData shape in grad descent loop, k=1', batchDataBack)
            self.bprop(batchDataBack)
        else: # For minibatch with size K
            running_error = 0
            for i in range(K):
                print( "batchData[i]", batchData[i].shape)
                self.fprop(batchData[i], batchTarget[i])
            print("grad loop multi data", batchData.shape)
            self.bprop(batchData)
            self.updateParams()
            
            
    '''
    def gradDescentMat(self, batchData, batchTarget):
        # Feed the entire data matrix in as input
        self.fprop(batchData, batchTarget)
        self.bprop(batchData, batchTarget)
    

        
    def dataSplit(self):
        '''
        train
        test
        valid
        '''
            
'''      
def earlyStopping(net):
    totalEpoch = 10 #may not be enough??
    for each epoch
        train
        valid
        test
        plot / print errors
'''



'      \ndef earlyStopping(net):\n    totalEpoch = 10 #may not be enough??\n    for each epoch\n        train\n        valid\n        test\n        plot / print errors\n'

In [22]:
x_i = trainData[0:1]
y_i = trainTarget[0:1].T


# !!! TODO: transpose all the targets in load data

#print("x_i.shape", x_i)
print("yi shape", y_i.shape)
nn = neuralNet(2, 5, 2, 1)

# d = 2
# dh = 5
# m =2
# n = 2

nn.gradDescentMat(x_i, y_i)

grad_W1 = nn.grad_W1[0][1]
oldErr = nn.errorRate(y_i)
print('oldErr', oldErr)

nnDebug = nn

nnDebug.W_1[0][1] += 1e-5

nnDebug.fprop(x_i, y_i)
newErr = nnDebug.errorRate(y_i)
print('newErr', newErr)
print('(newErr - oldErr)', (newErr - oldErr))

estimate = (newErr - oldErr) / 1e-5
ratio = estimate / grad_W1

print("                      ")
print("**********************")
print("estimate \n", estimate)
print("old gradient \n", grad_W1)
print("ratio \n", ratio )


yi shape (2, 1)
self.h_a  (5, 1)
h_s relu output  (5, 1)
self oa  (2, 1)
output softmax o_s (2, 1)
os (2, 1)
self.grad_oa (2, 1)
grad_W1 [[ 0.14644683 -0.09891492]
 [ 0.07620524 -0.05147148]
 [-0.          0.        ]
 [-0.08889365  0.06004165]
 [-0.          0.        ]]
grad_b1 [[ 0.17672248]
 [ 0.09195951]
 [-0.        ]
 [-0.10727106]
 [-0.        ]]
y.shape (2, 1)
oldErr [0.7142822]
self.h_a  (5, 1)
h_s relu output  (5, 1)
self oa  (2, 1)
output softmax o_s (2, 1)
y.shape (2, 1)
newErr [0.71428121]
(newErr - oldErr) [-9.8914875e-07]
                      
**********************
estimate 
 [-0.09891488]
old gradient 
 -0.09891492193859343
ratio 
 [0.99999953]


In [329]:
dh = 2
n = 5
d = 3

t1 = np.random.rand(2,5)
t2 = np.random.rand(5,3)
print(t1.shape)
print(t2.shape)
t = np.outer(t1, t2)
print(t.shape)

t1.reshape


m = 2
h_s_extend = np.repeat(t1 , m ,axis = 0)  # [n*m, dh]
h_s_extend = np.reshape(h_s_extend,[n, m, dh])    # [n,m,dh]
grad_oa = np.reshape(grad_oa, [n, m, 1])
print(h_s_extend.shape)


grad_ha_extend = np.repeat(grad_ha, d, axis = 0) #[dh*d, n]
grad_ha_extend = np.reshape(grad_ha_extend, [n, d, dh])
x = np.reshape(x, [n ,d, 1])
# !!! grad_W1 needs transpose


(2, 5)
(5, 3)
(10, 15)


NameError: name 'grad_oa' is not defined

In [267]:
class debugNet():
    
    def __init__(self, originalNet):
        self.copyNet = copy.deepcopy(originalNet)
        self.originalNet = originalNet
        self.idxList = ['W2', 'b2', 'W1', 'b1']
        self.originalParamList = {'W2': originalNet.W_2, 'b2': originalNet.b_2, 
                             'W1': originalNet.W_1, 'b1': originalNet.b_1}
        self.copyParamList = {'W2': self.copyNet.W_2, 'b2': self.copyNet.b_2, 
                              'W1': self.copyNet.W_1, 'b1': self.copyNet.b_1}
        self.originalGradientList = {'W2': originalNet.grad_W2, 'b2': originalNet.grad_b2,
                                    'W1': originalNet.grad_W1, 'b1': originalNet.grad_b1}
        
    def finiteGradCheck(self, sigma, x, y):
        #for i, param in enumerate(copyParamList):
        y = np.argmax(y)
        print("y at top of debug ", y)

        for key in self.idxList: #, param in self.copyParamList.items():
            param = self.copyParamList[key]
            print('key:', key, 'param:', param)
            
            '''if x.ndim == 1:
                originalShape = self.originalParamList[key].shape
                flatParams = np.ndarray.flatten(param)
                print('flattened parameters:', flatParams)
                for j, item in enumerate(flatParams):
                    print('param j:', item)
                    self.copyParamList[key][j] += sigma
                    self.copyNet.fprop(x, y)
                    originalScalar = self.originalParamList[key][j]
                    grad_estimate = ( self.copyNet.errorRate(y) - self.originalNet.errorRate(y)) / sigma
                    #print('self.originalGradientList', self.originalGradientList)
                    print("self.originalGradientList[key] ", self.originalGradientList[key])
                    originalGradient = self.originalGradientList[key][j]
                    ratio = originalGradient / grad_estimate
                    print('The ratio of calculated gradient and estimated gradient by perturbing {} gives {}.'.format(key, ratio))
                    # Set back
                    self.copyParamList[key][j] = originalScalar'''
            if 1==1:
                flatCopyParam = np.ndarray.flatten(param)
                flatOriginalParam = np.ndarray.flatten(self.originalParamList[key])
                originalShape = self.originalParamList[key].shape
                for j, _ in enumerate(flatCopyParam):
                    jRow = j % originalShape[0]
                    jCol = j // originalShape[0]
                    #print("original shape", originalShape)
                    #print("j % originalShape[0] = {}, j // originalShape[0] = {}".format(j % originalShape[0],j // originalShape[0]))
                    
                    
                    # Perturb
                    if param.ndim > 1:
                        self.copyParamList[key][jRow][jCol] += sigma
                    else:
                        self.originalParamList[key][jCol] +=sigma
                    self.copyNet.fprop(x, y)
                    
                    if param.ndim > 1:
                        originalScalar = self.originalParamList[key][jRow][jCol]
                    else:
                        originalScalar = self.originalParamList[key][jCol]
                    grad_estimate = ( self.copyNet.errorRate(y) - self.originalNet.errorRate(y)) / sigma
                    
                    if param.ndim > 1:
                        originalGradient = self.originalGradientList[key][jRow][jCol]
                    else:
                        originalGradient = self.originalGradientList[key][jCol]
                    ratio = originalGradient / grad_estimate
                    print('The ratio of calculated gradient and estimated gradient by perturbing {} gives \
                            {}.'.format(key, ratio))
                    #print('original gradient', originalGradient)
                    #print('grad_estimate ', grad_estimate)
                    # Set back
                    if param.ndim > 1:
                        self.copyParamList[key][jRow][jCol] = originalScalar
                    else:
                        self.copyParamList[key][jCol] = originalScalar
                

In [226]:
sigma = 1e-5
oldGradW_2 = nn.grad_W2
oldErr = nn.errorRate(np.argmax(trainTarget[0]))
print("oldGrad W2 ", oldGradW_2[0][0])
nn.W_2[0] += sigma
np.argmax(trainTarget[0])
estimate = (nn.errorRate(np.argmax(trainTarget[0])) - oldErr) / sigma
print("estimate ", estimate)
print("ratio ", oldGradW_2[0][0] / estimate)

y.shape ()


IndexError: tuple index out of range

In [80]:
# Do forward and backprop for one example

# Initialize net
print("input ", trainData[0:1][0])


nn = neuralNet(2, 1, 2)
nn.gradDescentLoop(trainData[0:1][0],  trainTarget[0], 1)

# Print the gradients

print('W1 gradient: \n{} \n\n b1 gradient:\n{}'.format(nn.grad_W1, nn.grad_b1))
print("softmax result: \n {}".format(nn.o_s.shape))

finiteGrad = debugNet(nn)

# single input add one dim hack
print("trainTarget[0:]", trainTarget[0:1].T.shape)

finiteGrad.finiteGradCheck(0.000001, trainData[0:1][0], trainTarget[0])





input  [ 0.82868251 -0.55971895]
check init weights
self.W_1  [[0.074175   0.03986865]]
self.W_2  [[ 0.28348563]
 [-0.72109409]]
relu output  [0.03915228]
output softmax o_s [0.50983163 0.49016837]
*** input to grad W2 *** 
self.grad_oa  [[-0.49016837]
 [ 0.49016837]]
self.h_s  [[0.03915228]]
**** input to W1 ***
self.grad_ha  [[-0.4924132]]
batchData  [[-0.4924132]]
grad_W1 [[-0.40805421  0.275613  ]]
grad_b1 [[-0.4924132]]
W1 gradient: 
[[-0.40805421  0.275613  ]] 

 b1 gradient:
[[-0.4924132]]
softmax result: 
 (2,)
trainTarget[0:] (2, 1)
y at top of debug  0
key: W2 param: [[ 0.28348563]
 [-0.72109409]]
original shape (2, 1)
j % originalShape[0] = 0, j // originalShape[0] = 0
relu output  [0.03915228]
output softmax o_s [0.50983164 0.49016836]
The ratio of calculated gradient and estimated gradient by perturbing W2 gives                             1.000000007793702.
original gradient -0.019191211103299022
grad_estimate  -0.019191210953728444
original shape (2, 1)
j % originalShape

/Users/vikuo/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:68: RuntimeWarning: divide by zero encountered in true_divide


### Part 2

> Display  the  gradients  for  both  methods (direct computation and finite difference) for a small network (e.g. $d = 2$ and $d_{h} = 2$) with random weights and for a single example.

### Part 3

> Add a hyperparameter for the minibatch size $K$ to allow computing the gradients on a minibatch of $K$ examples (in a matrix), by looping over the $K$ examples (this is a small addition to your previous code).

### Part 4

> Display the gradients for both methods (direct computation and finite difference) for a small network (e.g. $d = 2$ and $d_{h} = 2$) with random weights and for a minibatch with 10 examples (you can use examples from both classes from the two circles dataset).

### Part 5

> Train your neural network using gradient descent on the two circles dataset. Plot the decision regions for several different values of the hyperparameters (weight decay, number of hidden units, early stopping) so as to illustrate their effect on the capacity of the model.

In [9]:
#trainData = np.squeeze(trainData, axis = 2)

print(trainTarget[0:10].shape)

nn = neuralNet(2, 2, 2)
nn.gradDescentLoop((trainData[0:10]), trainTarget[0:10], 10)

(10, 2)
check init weights
self.W_1  [[ 0.37084291  0.37646992]
 [ 0.22274041 -0.45979189]]
self.W_2  [[ 0.49756088  0.35250494]
 [ 0.57126469 -0.4591249 ]]
batchData[i] (2,)
batchData[i] (2,)
batchData[i] (2,)
batchData[i] (2,)
batchData[i] (2,)
batchData[i] (2,)
batchData[i] (2,)
batchData[i] (2,)
batchData[i] (2,)
batchData[i] (2,)
grad loop multi data (10, 2)
self.grad_hs  (2,)
self.h_a  [-0.44909742  0.39922038]
npwhere  [0 1]
h_a_stack  [0 1]
grad_ha (2,)
batchData before grad_W1 (10, 2)


ValueError: shapes (2,) and (10,2) not aligned: 2 (dim 0) != 10 (dim 0)

### Part 6

> As a second step, copy your existing implementation to modify it to a new implementation that will use matrix calculus (instead of a loop) on batches of size $K$ to improve efficiency. **Take the matrix expressions in numpy derived in the first part, and adapt them for a minibatch of size $K$. Show in your report what you have modified (describe the former and new expressions with the shapes of each matrix).**

### Part 7

> Compare both implementations (with a loop and with matrix calculus) to check that they both give the same values for the gradients on the parameters, first for $K = 1$, then for $K = 10$. Display the gradients for both methods.

### Part 8

> Time how long an epoch takes on Fashion MNIST (1 epoch = 1 full traversal through the whole training set) for $K = 100$ for both versions (loop over a minibatch and matrix caluclus).

### Part 9

> Adapt your code to compute the error (proportion of misclassified examples) on the training set as well as the total loss on the training set during each epoch of the training procedure, and at the end of each epoch, it computes the error and average loss on the validation set and the test set. Display the 6 corresponding figures (error and average loss on train/valid/test), and write them in a log file.

### Part 10

> Train your network on the Fashion MNIST dataset. Plot the training/valid/test curves (error and loss as a function of the epoch number, corresponding to what you wrote in a file in the last question). Add to your report the curves obtained using your best hyperparameters, i.e. for which you obtained your best error on the validations et. We suggest 2 plots: the first one will plot the error rate (train/valid/test with different colors, show which color in a legend) and the other one for the averaged loss (on train/valid/test). You should be able to get less than 20% test error.